In [1]:
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import pandas as pd
import math
%matplotlib inline

In [10]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
val


,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx,letter
0,4,10,4,8,4,6,7,9,6,7,6,7,2,8,9,10,B
1,5,5,6,8,3,5,7,10,9,7,6,5,3,8,4,8,D
2,5,5,5,7,2,1,15,5,3,12,9,4,0,8,3,6,F
3,3,1,4,1,1,7,8,6,8,7,10,8,3,10,1,8,U
4,2,3,2,2,1,5,4,5,6,2,2,5,1,7,1,6,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4,7,5,5,5,7,7,5,8,6,5,9,3,8,6,9,E
3496,4,9,5,7,3,6,11,2,8,8,12,8,1,11,1,7,T
3497,4,5,5,8,2,9,15,1,6,5,11,9,0,8,0,8,T
3498,3,9,5,7,5,12,5,1,6,8,4,5,1,8,5,10,I


In [11]:
#we will one hot encode the labels
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

def encode_label(col):
  enc = LabelEncoder()
  enc.fit(col)
  enc_col = enc.transform(col)

  ohe = OneHotEncoder()
  encoded = ohe.fit(enc_col.reshape(-1,1))
  
  return encoded.transform(enc_col.reshape(-1,1)).toarray()

labels = train.pop('letter')
val_labels = val.pop('letter')
labels = encode_label(labels)
val_labels = encode_label(val_labels)

In [12]:
train = train.values.astype('float32')
val = val.values.astype('float32')

In [13]:
# for i in range(train.shape[1]):
#   m = np.mean(train[:, i])
#   s = np.std(train[:, i])
#   for j in range(train.shape[0]):
#     train[j, i] = (train[j, i]-m)/s

mean_x = train.mean().astype(np.float32)
std = train.std().astype(np.float32)
train = (train-mean_x)/std

val_mean = val.mean().astype(np.float32)
val_std = val.std().astype(np.float32)
val = (val-val_mean)/val_std

train = train.T
labels = labels.T
val = val.T
val_labels = val_labels.T

val


array([[-0.6596851 , -0.31614068, -0.31614068, ..., -0.6596851 ,
        -1.0032295 , -1.346774  ],
       [ 1.4015814 , -0.31614068, -0.31614068, ..., -0.31614068,
         1.058037  ,  0.7144926 ],
       [-0.6596851 ,  0.02740375, -0.31614068, ..., -0.31614068,
        -0.31614068, -1.346774  ],
       ...,
       [ 0.7144926 ,  0.7144926 ,  0.7144926 , ...,  0.7144926 ,
         0.7144926 ,  0.02740375],
       [ 1.058037  , -0.6596851 , -1.0032295 , ..., -2.0338628 ,
        -0.31614068, -2.0338628 ],
       [ 1.4015814 ,  0.7144926 ,  0.02740375, ...,  0.7144926 ,
         1.4015814 ,  0.7144926 ]], dtype=float32)

In [15]:
#for a particular layer
#n_x = number of inputs
#n_h = number of units
#n_y = number of outputs
 
#this function just initializes the parameters
def initialize_parameters(layers_dims):
  
  
  parameters = {}
  L = len(layers_dims) 
 
  for l in range(1, L):
    parameters["W" + str(l)] = nr.randn(layers_dims[l], layers_dims[l-1]) * 0.0001
    parameters["b" + str(l)] = np.zeros((layers_dims[l], 1))
 
  return parameters

In [16]:
def relu(Z):
  
  return np.maximum(0, Z), Z
#these are the activation functions
#while returning z we will call it the activation_cache
#we will also return (A_prev, W, b) as linear_cache

In [17]:
def forward_prop(A_prev, W, b, activation):
 
  
  Z = np.dot(W, A_prev) + b
  A, activation_cache = relu(Z)
  linear_cache = (A_prev, W, b)
 
  cache = (linear_cache, activation_cache)
 
  return A, cache
 
def L_nn_forward(X, parameters):
  caches = []
  L = len(parameters) // 2
  #print(f'X shape: {X.shape}')
  A = X.reshape(X.shape[0], -1)
 
  for l in range(1, L+1): #check
    A_prev = A
    #print(f"A_prev shape = {A_prev.shape}")
    W = parameters['W' + str(l)]
    #print(f'W{l}: {W.shape}')
    b = parameters['b' + str(l)]
    #print(f'b{l}: {b.shape}')
    A, cache = forward_prop(A_prev, W, b, 'relu')
    #print(f'A: {A.shape}')
    caches.append(cache)
    
  
  return A, caches

In [18]:
parameters = initialize_parameters([7,5,7])
X = np.array([1,2,3,4,5,6,7]).reshape(-1,1)
A, caches = L_nn_forward(X, parameters) 


In [19]:
linear = []
activation = []
for tup in caches:
    linear.append(tup[0])
    activation.append(tup[1])

for x in linear:
    print(x[0].shape, x[1].shape, x[2].shape)

(7, 1) (5, 7) (5, 1)
(5, 1) (7, 5) (7, 1)


In [20]:
def Softmax(A_last):
    for i in range(A_last.shape[1]):
        # sum = 0
        # for x in A_last[:,i]:
        #     sum += np.exp(x)
        for j in range(A_last.shape[0]):
            A_last[j,i] = np.exp(A_last[j,i]) / np.sum(np.exp(A_last[:,i]))
    return A_last

In [21]:
A = Softmax(A)

In [22]:
#now for cost function
Y = np.array([1,0,0,1,1,0,1])

def compute_cost(A_last, Y):
  m = Y.shape[1]

  cost = (-1/m)*(np.sum(Y*np.log(A_last)))

  return np.squeeze(cost)


In [23]:
print(compute_cost(A, Y.reshape(-1,1)))

8.044931094244552


In [24]:
#now for backward
#dZ = dA * g'(Z)
#this function calculates dZ, dA_prev, db and dW given dA
#different for sigmoid and relu (2 extra functions)
#linear_cache has the values = A_prev, W, b (used to compute Z)
#activation_cache has the values Z

def sigmoid_backward(dA, activation_cache):
  Z = activation_cache
  return dA*(Z)*(1-Z)

def relu_backward(dA, activation_cache):
  Z = activation_cache
  diff = []
  diff.append( 1 if Z.any()>0 else 0)
  diff = np.array(diff)
  return dA*diff


def linear_acti_backward(dA, cache, activation):
  #cache because we wil need the linear part and the activation part 
  #for the computation

  linear_cache, activation_cache = cache #(A,W,b), Z
  A_prev, W, b = linear_cache
  m = A_prev.shape[1]

  dZ = relu_backward(dA, activation_cache)
  #print(f'dA: {dA.shape}, dZ: {dZ.shape}, A_prev: {A_prev.shape}')
  dW = (1/m)*(np.dot(dZ, A_prev.T))
  #print(f'dW: {dW.shape}')
  db =  (1/m)*(np.sum(dZ, axis = 1, keepdims = True))
  dA_prev = np.dot(W.T, dZ)
  #print(f'dA_prev: {dA_prev.shape}')
  return dA_prev, dW, db


In [25]:
def L_Backward(A_last, Y, caches):
    grads = {}
    L = len(caches) #number of layers
    m = A_last.shape[1]
    Y = Y.reshape(A_last.shape)

    dA_last = -(Y/A_last) + ((1-Y)/(1-A_last))
    current_cache = caches[L-1]
    grads["dA" + str(L-1)],grads["dW" + str(L)], grads["db" + str(L)] = linear_acti_backward(dA_last, current_cache, 'relu')
    #print(f'dA2: {dA_last.shape}, dA1: {grads["dA" + str(L-1)].shape}')

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev = grads["dA" + str(l+1)]
        A = current_cache[0][0]
        W = current_cache[0][1]
        b = current_cache[0][2]
        Z = current_cache[1][0]
        dZ = relu_backward(dA_prev, Z)
        grads["dW" + str(l+1)] =  (1/dA_prev.shape[1])*(np.dot(dZ, A.T))
        grads["db" + str(l+1)] = (1/dA_prev.shape[1])*(np.sum(dZ, axis = 1, keepdims = True))
        grads["dA" + str(l)] = np.dot(W.T, dZ)

    return grads


In [26]:
grads = L_Backward(A, Y.reshape(-1,1), caches)

In [27]:
def update_parameters(parameters, grads, learning_rate):

  L = len(parameters) // 2

  for l in range(L):
    parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate)*(grads["dW" + str(l+1)])
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate)*(grads["db" + str(l+1)])

  return parameters

In [34]:
#we will create a model in which we will execute all these gunctions
#l is the number of layers and layers_dims is the dimension of the layers
def deepnnmodel(X, Y,val, val_labels, layer_dims = [16, 26], batch_size = 500, learning_rate = 0.005, num_iterations =40):

  np.random.seed(1)
  costs = []
  val_costs = []
  parameters = initialize_parameters(layer_dims)

  for i in range(num_iterations):

    for j in range(0, X.shape[1], batch_size):
      X_train = X[:,j:j+batch_size]
      Y_train = Y[:,j:j+batch_size]
      
      A_last, caches = L_nn_forward(X_train, parameters)
      A_last = Softmax(A_last)
      cost = compute_cost(A_last, Y_train)

      grads =  L_Backward(A_last, Y_train, caches)
      #print(X_train)
      #print(f"""A_last: {A_last}
      #      cost: {cost}
      #     grads: {grads}""")

      parameters = update_parameters(parameters, grads, learning_rate)
        
      #for val
    val_x_train = val
    val_y_train = val_labels
    val_A_last, val_caches = L_nn_forward(val_x_train, parameters)
    val_A_last = Softmax(val_A_last)
    val_cost = compute_cost(val_A_last, val_y_train)


    print(f'Epoch {i+1}: loss: {compute_cost(A_last, Y_train)}, val_loss = {compute_cost(val_A_last, val_y_train)}')
    costs.append(compute_cost(A_last, Y_train))
    val_costs.append(compute_cost(val_A_last, val_y_train))
    
  return parameters, costs



In [35]:
params, cos = deepnnmodel(train, labels, val, val_labels)

Epoch 1: loss: 3.0454581028112235, val_loss = 3.0459110478827913
Epoch 2: loss: 2.9547331637847574, val_loss = 2.9608240294785815
Epoch 3: loss: 2.90923924032198, val_loss = 2.917629194726831
Epoch 4: loss: 2.8794492692893674, val_loss = 2.8891672182472816
Epoch 5: loss: 2.8568386743736083, val_loss = 2.8668792650093473
Epoch 6: loss: 2.838288479222592, val_loss = 2.8478208708269266
Epoch 7: loss: 2.821890678522472, val_loss = 2.830567799492097
Epoch 8: loss: 2.806181063543579, val_loss = 2.814764275718791
Epoch 9: loss: 2.7915883736372553, val_loss = 2.8001550473070433
Epoch 10: loss: 2.7779932850734355, val_loss = 2.7866116423471574
Epoch 11: loss: 2.7653727986962524, val_loss = 2.773879234448845
Epoch 12: loss: 2.753784585536644, val_loss = 2.7620971771123277
Epoch 13: loss: 2.74310854225081, val_loss = 2.751015791890723
Epoch 14: loss: 2.7329017615991633, val_loss = 2.7406740083474337
Epoch 15: loss: 2.7234798545538523, val_loss = 2.7310938914115743
Epoch 16: loss: 2.71472371308283

In [36]:
test =  pd.read_csv('test.csv')
test

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,3,5,4,4,2,7,8,5,4,7,7,7,6,9,2,5
1,4,6,6,6,4,7,5,4,4,8,3,7,3,7,4,8
2,3,7,4,5,3,8,7,7,6,7,6,7,3,8,4,7
3,3,6,4,4,2,5,12,7,3,12,6,3,1,10,4,7
4,3,7,4,5,3,7,6,6,6,6,5,9,2,9,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,2,3,4,1,2,6,8,1,6,10,6,9,3,8,2,8
3496,6,9,8,7,5,9,8,1,8,14,5,6,5,8,5,9
3497,4,11,6,8,6,6,10,4,5,10,9,4,4,10,4,7
3498,1,0,2,1,0,7,7,11,0,5,6,8,4,8,0,8


In [37]:
test = test.values.astype('float32')

In [38]:
test = test.T

In [39]:
W1 = params["W1"]
b1 = params["b1"]



In [40]:
A1 = np.dot(W1, test) + b1
A1, _ = relu(A1)

In [41]:
A1 = Softmax(A1)
A1.shape

(26, 3500)

In [42]:
maxi = []
for i in range(A1.shape[1]):
    maxi.append(A1[:,i].argmax())

In [43]:
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [44]:
preds = []
for i in maxi:
    preds.append(letters[i])

In [45]:
preds[:200]

['C',
 'G',
 'C',
 'P',
 'G',
 'E',
 'V',
 'H',
 'Q',
 'C',
 'C',
 'G',
 'Q',
 'H',
 'G',
 'U',
 'T',
 'G',
 'G',
 'Q',
 'G',
 'T',
 'T',
 'Z',
 'E',
 'Q',
 'Z',
 'C',
 'C',
 'C',
 'T',
 'E',
 'G',
 'T',
 'G',
 'G',
 'E',
 'E',
 'H',
 'C',
 'G',
 'G',
 'E',
 'C',
 'Q',
 'E',
 'G',
 'Z',
 'G',
 'T',
 'E',
 'E',
 'C',
 'G',
 'Z',
 'G',
 'T',
 'H',
 'G',
 'H',
 'E',
 'H',
 'U',
 'C',
 'E',
 'H',
 'Z',
 'G',
 'Y',
 'T',
 'Q',
 'T',
 'H',
 'T',
 'T',
 'T',
 'E',
 'C',
 'H',
 'V',
 'Q',
 'T',
 'Q',
 'Q',
 'V',
 'E',
 'C',
 'Q',
 'C',
 'E',
 'E',
 'E',
 'G',
 'G',
 'H',
 'E',
 'H',
 'H',
 'E',
 'G',
 'E',
 'V',
 'G',
 'T',
 'C',
 'G',
 'E',
 'G',
 'Q',
 'G',
 'H',
 'H',
 'E',
 'Z',
 'H',
 'E',
 'E',
 'P',
 'Q',
 'H',
 'G',
 'T',
 'Z',
 'V',
 'C',
 'G',
 'G',
 'C',
 'E',
 'E',
 'C',
 'G',
 'G',
 'G',
 'E',
 'E',
 'E',
 'Q',
 'Q',
 'G',
 'M',
 'E',
 'U',
 'G',
 'K',
 'Q',
 'G',
 'Q',
 'E',
 'E',
 'G',
 'T',
 'C',
 'C',
 'G',
 'E',
 'E',
 'C',
 'E',
 'V',
 'G',
 'C',
 'C',
 'E',
 'E',
 'C',
 'V'

In [46]:
predictions = pd.DataFrame({"ID": list(range(1,len(preds)+1)),
                           "Label": preds})
predictions.to_csv("test_predictions", index = False)